<a href="https://colab.research.google.com/github/Aswin-Kumar66/Pyspark_Practise/blob/main/Pyspark_Practise_3___UDF_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=65c5fcca19aec84490e81bcf035c5be19278b6709d46a3881b28416496f5412d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

## Question 5

In [126]:
df = spark.read.csv('student-mat.csv' , header = True , inferSchema = True)

In [127]:
df.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|
|    GP|  F| 17|    

In [128]:
df = df.select(df.columns[df.columns.index('school'):df.columns.index('guardian')+1])

In [129]:
df.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|  mother|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| at_home|   other|    course|  father|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| at_home|   other|     other|  mother|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  health|services|      home|  mother|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   other|   other|      home|  father|
|    GP|  M| 16|      U|    LE3|      T|   4|   3|services|   other|reputation|  mother|
|    GP|  M| 16|      U|    LE3|      T|   2|   2|   other|   other|      home|  mother|
|    GP|  F| 17|      U|    GT3|      A|   4|   4|   other| teacher|      home|  mother|
|    GP|  M| 15|     

In [140]:
def cap(x):
  return x.capitalize()

capitalize = F.udf(lambda x : cap(x))
df = df.withColumn('Mjob' , capitalize(F.col('Mjob'))).withColumn('Fjob' , capitalize(F.col('Fjob')))

In [142]:
df.printSchema()

root
 |-- school: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- famsize: string (nullable = true)
 |-- Pstatus: string (nullable = true)
 |-- Medu: integer (nullable = true)
 |-- Fedu: integer (nullable = true)
 |-- Mjob: string (nullable = true)
 |-- Fjob: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- guardian: string (nullable = true)



In [137]:
df.tail(5)

[Row(school='MS', sex='M', age=20, address='U', famsize='LE3', Pstatus='A', Medu=2, Fedu=2, Mjob='Services', Fjob='Services', reason='course', guardian='other'),
 Row(school='MS', sex='M', age=17, address='U', famsize='LE3', Pstatus='T', Medu=3, Fedu=1, Mjob='Services', Fjob='Services', reason='course', guardian='mother'),
 Row(school='MS', sex='M', age=21, address='R', famsize='GT3', Pstatus='T', Medu=1, Fedu=1, Mjob='Other', Fjob='Other', reason='course', guardian='other'),
 Row(school='MS', sex='M', age=18, address='R', famsize='LE3', Pstatus='T', Medu=3, Fedu=2, Mjob='Services', Fjob='Other', reason='course', guardian='mother'),
 Row(school='MS', sex='M', age=19, address='U', famsize='LE3', Pstatus='T', Medu=1, Fedu=1, Mjob='Other', Fjob='At_home', reason='course', guardian='father')]

In [144]:
def legal(x):
  if x > 17 :
    return 'Yes'
  else :
    return 'No'

convert = F.udf(lambda x : legal(x))
df.withColumn('legal_drinker' , convert(F.col('age'))).show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+-------------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|legal_drinker|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+-------------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| At_home| Teacher|    course|  mother|          Yes|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| At_home|   Other|    course|  father|           No|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| At_home|   Other|     other|  mother|           No|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  Health|Services|      home|  mother|           No|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   Other|   Other|      home|  father|           No|
|    GP|  M| 16|      U|    LE3|      T|   4|   3|Services|   Other|reputation|  mother|           No|
|    GP|  M| 16|      U|    LE3|      T|   2|   2|   Other|   Other|     

In [146]:
def mul(x):
  return x*10

mul_10 = F.udf(lambda x : mul(x))
df.select('*' , mul_10(F.col('age')) , mul_10(F.col('Medu')), mul_10(F.col('Fedu')) ).show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+-------------+--------------+--------------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|<lambda>(age)|<lambda>(Medu)|<lambda>(Fedu)|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+-------------+--------------+--------------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| At_home| Teacher|    course|  mother|          180|            40|            40|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| At_home|   Other|    course|  father|          170|            10|            10|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| At_home|   Other|     other|  mother|          150|            10|            10|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  Health|Services|      home|  mother|          150|            40|            20|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   Other|   Other| 